In [1]:
from keras.models import Model, load_model
from keras.layers import Input, Dropout, Dense, Embedding, SpatialDropout1D, concatenate, BatchNormalization, Flatten
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
from keras.models import Model
from keras.losses import mean_squared_error as mse_loss

from keras import optimizers
from keras.optimizers import RMSprop, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np # linear algebra
import pandas as pd #

Using TensorFlow backend.


In [ ]:
ques_info=pd.read_csv('question_info.txt',sep='\s+',names=['qid','qtime','qtitle','qtitlec','qinfo','qinfoc','qtopic'])
ques_info=ques_info[['qid','qtitle','qinfo']]

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

embed_size=64
tokenizer=Tokenizer(lower=False,char_level=False,split=',')
tokenizer.fit_on_texts(ques_info['qtitle'].append(ques_info['qinfo']))

In [7]:
import pickle
with open('char_tokenizer.pickle', 'wb') as f:
      pickle.dump(tokenizer, f)

In [6]:
import pickle

def load_obj(name ):
    with open( name + '.pkl', 'rb') as f:
        return pickle.load(f)
topic_w2v=load_obj('single_word_vectors_64d')

max_features=len(tokenizer.word_index)+1
char_embedding=np.zeros((max_features,embed_size))
num_oov=0
for word in tokenizer.word_index:
    try:
        char_embedding[tokenizer.word_index[word]]=topic_w2v[word]['vector']
    except:
        print(tokenizer.word_index[word])
        print(word)
        np.random.seed(1997)
        char_embedding[tokenizer.word_index[word]]=np.random.normal(1,1,64)
        num_oov+=1
print('oov num is '+str(num_oov))

16
1
oov num is 1


In [8]:
np.save("esim_char_embedding.npy", char_embedding)

In [ ]:
#####################################一下为复赛需要运行的代码

In [2]:
ques_info=pd.read_csv('question_info.txt',sep='\s+',names=['qid','qtime','qtitle','qtitlec','qinfo','qinfoc','qtopic'])
ques_info=ques_info[['qid','qtitle','qinfo']]

In [3]:
train=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
test=pd.read_csv('invite_info_evaluate_2_0926.txt',sep='\s+',names=['qid','uid','time'])
all_data=pd.concat([train[['qid','uid']],test[['qid','uid']]])

In [4]:
last_answer=pd.read_hdf('last_answerlist_6.h5', key='data')

In [5]:
last_answer=last_answer[['qid','uid','qid_1']]
last_answer=last_answer.drop_duplicates(['qid','uid'])

In [6]:
all_data=all_data.merge(ques_info,on=['qid'],how='left')
all_data=all_data.merge(last_answer,on=['qid','uid'],how='left')

In [7]:
ques_info.columns=['qid_1','qtitle_1','qinfo_1']
all_data=all_data.merge(ques_info,on=['qid_1'],how='left')

In [8]:
all_data=all_data.fillna('-1')

In [9]:
import pickle
with open('char_tokenizer.pickle', 'rb') as f:
      tokenizer = pickle.load(f)

In [10]:
qtitle  =tokenizer.texts_to_sequences(all_data['qtitle'])
qinfo=tokenizer.texts_to_sequences(all_data['qinfo'])
qtitle_1  =tokenizer.texts_to_sequences(all_data['qtitle_1'])

In [11]:
qtitle=pad_sequences(qtitle,maxlen=30,value=0)
qinfo=pad_sequences(qinfo,maxlen=50,value=0)
qtitle_1=pad_sequences(qtitle_1,maxlen=30,value=0)

In [13]:
np.save("qtitle.npy", qtitle)
np.save("qinfo.npy", qinfo)
np.save("qtitle_1.npy", qtitle_1)

In [ ]:
##########################################################################

In [2]:
ques_info=pd.read_csv('question_info.txt',sep='\s+',names=['qid','qtime','qtitle','qtitlec','qinfo','qinfoc','qtopic'])
ques_info=ques_info[['qid','qtopic']]
last_answer=pd.read_hdf('last_answerlist_6_topic.h5', key='data')
train=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
test=pd.read_csv('invite_info_evaluate_1.txt',sep='\s+',names=['qid','uid','time'])
train=train[['qid','uid']]
test=test[['qid','uid']]
all_data=pd.concat([train,test])
member_info=pd.read_csv('member_info.txt',sep='\s+',names=['uid','sex','key_word','num_level','hot_level','regis_type','regis_platform',
                                                          'look_freq','a','b','c','d','e','A','B','C','D','E','salt','l_topic','topic_n'])
member_info=member_info[['uid','l_topic','topic_n']]
from tqdm import tqdm
tqdm.pandas()

In [3]:
def split_topic(x):
    x=x.split(',')
    return ','.join([i.split(':')[0] for i in x])
member_info['u_topic']=member_info['topic_n'].apply(lambda x:split_topic(x))
def split_weight(x):
    if x!='-1':
        x=x.split(',')
        return [float(i.split(':')[1]) for i in x]
    else:return [0]*10
member_info['w_topic']=member_info['topic_n'].apply(lambda x:split_weight(x))
member_info.loc[member_info['l_topic']=='-1','l_topic']=member_info[member_info['l_topic']=='-1']['u_topic']

In [5]:
last_answer['last_6_tpoic']=last_answer['last_6_tpoic'].progress_apply(lambda x:','.join(x))

100%|██████████| 7573780/7573780 [00:18<00:00, 413189.01it/s]


In [6]:
all_data=all_data.merge(ques_info,on='qid',how='left')
all_data=all_data.merge(member_info,on='uid',how='left')
all_data=all_data.merge(last_answer,on=['qid','uid'],how='left')

In [7]:
all_data=all_data.fillna('-1')

In [9]:
import pickle
with open('esim_topic_tokenizer.pickle', 'rb') as f:
      tokenizer = pickle.load(f)

In [13]:
all_data['last_6_tpoic']=all_data['last_6_tpoic'].astype(str)

In [ ]:
u_topic  =tokenizer.texts_to_sequences(all_data['u_topic'])
qtopic=tokenizer.texts_to_sequences(all_data['qtopic'])
l_topic  =tokenizer.texts_to_sequences(all_data['l_topic'])

In [14]:
last_6_tpoic  =tokenizer.texts_to_sequences(all_data['last_6_tpoic'])

In [16]:
last_6_tpoic=pad_sequences(last_6_tpoic,maxlen=50,value=0)
l_topic=pad_sequences(l_topic,maxlen=50,value=0)
u_topic=pad_sequences(u_topic,maxlen=10,value=0)
qtopic_10=pad_sequences(qtopic,maxlen=10,value=0)
qtopic_50=pad_sequences(qtopic,maxlen=50,value=0)

In [29]:
w_topic=pad_sequences(all_data['w_topic'].tolist(),maxlen=10,value=0, dtype='float32')

In [30]:
w_topic.shape

(10630845, 10)

In [31]:
w_topic = np.array(pd.DataFrame(w_topic).replace([np.inf, -np.inf], 1).fillna(1))

In [33]:
np.save("last_6_tpoic.npy", last_6_tpoic)
np.save("l_topic.npy", l_topic)
np.save("u_topic.npy", u_topic)
np.save("qtopic_10.npy", qtopic_10)
np.save("qtopic_50.npy", qtopic_50)
np.save("w_topic.npy", w_topic)